# Reconhecimento Óptico de Caracteres (OCR) com Detecção de Contorno

Este notebook demonstra um fluxo de trabalho completo de OCR em uma imagem de amostra. O processo começa carregando um modelo de rede neural convolucional (CNN) pré-treinado, projetado para reconhecer caracteres individuais. Em seguida, a imagem de entrada é pré-processada para otimizar a detecção de caracteres. A etapa principal é a segmentação de caracteres, que usa a detecção de contorno do OpenCV para isolar cada caractere na imagem. Finalmente, cada caractere segmentado é preparado e passado para o modelo para previsão. O notebook conclui exibindo o texto reconhecido e visualizando os resultados com caixas delimitadoras e rótulos de caracteres na imagem original.

In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import json
from pathlib import Path
import cv2

In [2]:
def find_project_root(markers=("pyproject.toml", ".git")) -> Path:
    """Localiza a pasta raiz do projeto.

    Args:
        markers (tuple, optional): Uma tupla de nomes de arquivos ou pastas que marcam a raiz do projeto.
                                     O padrão é ("pyproject.toml", ".git").

    Returns:
        Path: O objeto Path para o diretório raiz do projeto.
    """
    p = Path.cwd().resolve()
    for parent in [p, *p.parents]:
        if any((parent / m).exists() for m in markers):
            return parent
    return p

ROOT = find_project_root()
print(f"Raiz do projeto: {ROOT}")

### Configurações

Define os caminhos para os arquivos de dados, o modelo treinado, a imagem de teste e as classes, além de definir as dimensões das imagens.

In [3]:
# Configurações
DATA_DIR = ROOT / "data"
MODEL_PATH = DATA_DIR / "processed" / "modelo_ocr_simbolos.keras"
IMAGE_PATH = DATA_DIR / "raw" / "ocr_test_image.png"
CLASS_NAMES_PATH = DATA_DIR / "raw" / "class_names.json"

IMG_HEIGHT = 64
IMG_WIDTH = 64

### Carregamento do Modelo

Carrega o modelo de OCR treinado e exibe um resumo da sua arquitetura.

In [4]:
# Carrega o modelo treinado
print("Carregando modelo...")
model = tf.keras.models.load_model(MODEL_PATH)
print("✓ Modelo carregado com sucesso!")
model.summary()

### Carregamento dos Nomes das Classes

Carrega os nomes das classes (caracteres) do arquivo JSON.

In [5]:
# Carrega os nomes das classes
with open(CLASS_NAMES_PATH, 'r', encoding='utf-8') as f:
    class_names = json.load(f)
print(f"✓ {len(class_names)} classes carregadas")
print(f"Exemplos: {class_names[:5]}")

In [6]:
def unicode_to_char(unicode_str: str) -> str:
    """Converte uma string unicode no formato 'U+XXXX' para o caractere correspondente.

    Args:
        unicode_str (str): A string no formato 'U+XXXX'.

    Returns:
        str: O caractere correspondente ou '?' em caso de erro.
    """
    try:
        codepoint = int(unicode_str.replace('U+', ''), 16)
        return chr(codepoint)
    except:
        return '?'

### Carregamento da Imagem de Teste

Carrega e exibe a imagem que será utilizada para o OCR.

In [7]:
# Carrega e exibe a imagem original
print(f"Carregando imagem: {IMAGE_PATH}")
original_image = Image.open(IMAGE_PATH)

plt.figure(figsize=(12, 4))
plt.imshow(original_image, cmap='gray')
plt.title('Imagem Original para OCR')
plt.axis('off')
plt.show()

print(f"Dimensões: {original_image.size}")
print(f"Modo: {original_image.mode}")

### Pré-processamento da Imagem

A imagem é convertida para escala de cinza, binarizada usando um threshold adaptativo e, se necessário, invertida para garantir que o texto seja preto e o fundo branco, que é o formato esperado pelo modelo.

In [8]:
def preprocess_image_for_ocr(image: Image.Image) -> np.ndarray:
    """ Pré-processa a imagem para o OCR.

    Aplica as seguintes etapas:
    - Converte a imagem para escala de cinza.
    - Aplica um threshold adaptativo para binarizar a imagem.
    - Inverte as cores, se necessário, para garantir um fundo branco e texto preto.

    Args:
        image (Image.Image): A imagem de entrada.

    Returns:
        np.ndarray: A imagem pré-processada como um array NumPy.
    """
    # Converte para escala de cinza
    if image.mode != 'L':
        image = image.convert('L')

    # Converte para numpy array
    img_array = np.array(image)

    # Aplica threshold adaptativo para melhor binarização
    img_binary = cv2.adaptiveThreshold(
        img_array, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )

    # Verifica se precisa inverter (modelo espera fundo branco, texto preto)
    # Se a média é baixa, a imagem está com fundo escuro
    if np.mean(img_binary) < 127:
        img_binary = 255 - img_binary

    return img_binary

In [9]:
# Pré-processa a imagem
print("Pré-processando imagem...")
processed_image = preprocess_image_for_ocr(original_image)

plt.figure(figsize=(12, 4))
plt.imshow(processed_image, cmap='gray')
plt.title('Imagem Pré-processada (Binarizada)')
plt.axis('off')
plt.show()

### Segmentação de Caracteres

Utiliza a detecção de contornos para segmentar (isolar) cada caractere individual na imagem pré-processada. Os caracteres são então ordenados da esquerda para a direita.

In [10]:
def segment_characters(image_array: np.ndarray) -> list:
    """Segmenta os caracteres individuais da imagem binarizada.

    Esta função utiliza o algoritmo de detecção de contornos do OpenCV para encontrar os limites
    de cada caractere. Os contornos são então filtrados para remover ruídos e ordenados 
    da esquerda para a direita com base em sua posição no eixo x.

    Args:
        image_array (np.ndarray): Uma matriz NumPy representando a imagem binarizada 
                                (fundo branco, texto preto).

    Returns:
        list: Uma lista de tuplas, onde cada tupla contém:
              - Uma tupla (x, y, w, h) representando a caixa delimitadora do caractere.
              - Uma matriz NumPy da imagem do caractere segmentado com preenchimento.
    """
    # Encontra contornos na imagem. A imagem é invertida (255 - image_array) 
    # porque a função `findContours` espera que os objetos sejam brancos em um fundo preto.
    contours, _ = cv2.findContours(
        255 - image_array,  # Inverte para findContours
        cv2.RETR_EXTERNAL, # Recupera apenas os contornos externos
        cv2.CHAIN_APPROX_SIMPLE # Comprime segmentos horizontais, verticais e diagonais e deixa apenas seus pontos finais
    )

    # Filtra e ordena os contornos da esquerda para a direita
    char_regions = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Filtra contornos muito pequenos que provavelmente são ruído
        if w > 1 and h > 1:
            char_regions.append((x, y, w, h))

    # Ordena as regiões de caracteres com base na coordenada x (da esquerda para a direita)
    char_regions = sorted(char_regions, key=lambda r: r[0])

    # Extrai as regiões da imagem para cada caractere
    segmented_chars = []
    for (x, y, w, h) in char_regions:
        # Adiciona um preenchimento (padding) ao redor de cada caractere para garantir 
        # que ele não seja cortado
        padding = 5
        x1 = max(0, x - padding)
        y1 = max(0, y - padding)
        x2 = min(image_array.shape[1], x + w + padding)
        y2 = min(image_array.shape[0], y + h + padding)

        char_img = image_array[y1:y2, x1:x2]
        segmented_chars.append(((x, y, w, h), char_img))

    return segmented_chars

In [11]:
# Segmenta caracteres
print("Segmentando caracteres...")
segmented_chars = segment_characters(processed_image)
print(f"✓ {len(segmented_chars)} caracteres detectados")

# Visualiza caracteres segmentados
if len(segmented_chars) > 0:
    n_chars = len(segmented_chars)
    cols = min(10, n_chars)
    rows = (n_chars + cols - 1) // cols

    plt.figure(figsize=(15, 2 * rows))
    for i, (bbox, char_img) in enumerate(segmented_chars):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(char_img, cmap='gray')
        plt.title(f'#{i+1}')
        plt.axis('off')
    plt.suptitle('Caracteres Segmentados')
    plt.tight_layout()
    plt.show()
else:
    print("⚠ Nenhum caractere foi detectado!")

### Preparação para Predição

Redimensiona e centraliza cada caractere segmentado para o formato esperado pelo modelo (64x64 pixels), preparando-o para a predição.

In [12]:
def prepare_char_for_prediction(char_image: np.ndarray) -> np.ndarray:
    """Prepara a imagem de um caractere segmentado para a predição do modelo.

    Esta função pega a imagem de um único caractere, a centraliza em uma tela quadrada 
    (canvas) branca, redimensiona para as dimensões esperadas pelo modelo (64x64) e 
    remodela o array para o formato de entrada do modelo (1, 64, 64, 1).

    Args:
        char_image (np.ndarray): A imagem do caractere segmentado (fundo branco).

    Returns:
        np.ndarray: A imagem preparada, pronta para ser usada como entrada para o 
                    modelo de previsão.
    """
    # Cria um canvas quadrado branco do tamanho do maior lado da imagem
    size = max(char_image.shape)
    canvas = np.ones((size, size), dtype=np.uint8) * 255

    # Calcula os deslocamentos para centralizar o caractere no canvas
    y_offset = (size - char_image.shape[0]) // 2
    x_offset = (size - char_image.shape[1]) // 2
    # Coloca o caractere no centro do canvas
    canvas[y_offset:y_offset+char_image.shape[0],
           x_offset:x_offset+char_image.shape[1]] = char_image

    # Redimensiona o canvas para as dimensões de entrada do modelo (64x64)
    # A interpolação LANCZOS4 é usada por ser boa para reduzir o tamanho de imagens
    resized = cv2.resize(canvas, (IMG_WIDTH, IMG_HEIGHT),
                        interpolation=cv2.INTER_LANCZOS4)

    # Adiciona as dimensões do lote (batch) e do canal (channel) para corresponder ao formato de 
    # entrada do modelo: (1, 64, 64, 1)
    # A normalização (divisão por 255) não é feita aqui porque o modelo possui uma camada de 
    # "Rescaling(1./255)" como primeira camada.
    prepared = resized.reshape(1, IMG_HEIGHT, IMG_WIDTH, 1)

    return prepared

In [13]:
# Realiza OCR em cada caractere
print("\nRealizando OCR...")
results = []

for i, (bbox, char_img) in enumerate(segmented_chars):
    # Prepara para predição
    input_array = prepare_char_for_prediction(char_img)

    # Predição
    predictions = model.predict(input_array, verbose=0)
    predicted_idx = np.argmax(predictions[0])
    confidence = predictions[0][predicted_idx]

    # Converte para caractere
    unicode_class = class_names[predicted_idx]
    character = unicode_to_char(unicode_class)

    results.append({
        'index': i,
        'bbox': bbox,
        'unicode': unicode_class,
        'character': character,
        'confidence': confidence,
        'image': char_img
    })

    print(f"Caractere {i+1}: '{character}' ({unicode_class}) - Confiança: {confidence:.4f}")

# Texto reconhecido
recognized_text = ''.join([r['character'] for r in results])
print(f"\n{'='*60}")
print(f"TEXTO RECONHECIDO: {recognized_text}")
print(f"{'='*60}")

In [14]:
# Visualização detalhada dos resultados
if len(results) > 0:
    n_results = len(results)
    cols = min(8, n_results)
    rows = (n_results + cols - 1) // cols

    plt.figure(figsize=(16, 3 * rows))
    for i, result in enumerate(results):
        # Imagem preparada para o modelo
        prepared = prepare_char_for_prediction(result['image'])

        plt.subplot(rows, cols, i + 1)
        plt.imshow(prepared.squeeze(), cmap='gray')
        plt.title(
            f"'{result['character']}'\n"
            f"{result['unicode']}\n"
            f"Conf: {result['confidence']:.2%}",
            fontsize=10
        )
        plt.axis('off')

    plt.suptitle(
        f'Resultados do OCR: "{recognized_text}"',
        fontsize=14, fontweight='bold'
    )
    plt.tight_layout()
    plt.show()

In [15]:
# Imagem com anotações
annotated = cv2.cvtColor(processed_image.copy(), cv2.COLOR_GRAY2RGB)

for result in results:
    x, y, w, h = result['bbox']
    # Desenha retângulo
    cv2.rectangle(annotated, (x, y), (x+w, y+h), (0, 255, 0), 2)
    # Adiciona texto
    cv2.putText(
        annotated, result['character'],
        (x, y-5), cv2.FONT_HERSHEY_SIMPLEX,
        0.7, (255, 0, 0), 2
    )

plt.figure(figsize=(15, 5))
plt.imshow(annotated)
plt.title(f'OCR Completo: "{recognized_text}"')
plt.axis('off')
plt.show()

In [16]:
# Salva resultados
output_data = {
    'recognized_text': recognized_text,
    'total_characters': len(results),
    'average_confidence': float(np.mean([r['confidence'] for r in results])),
    'characters': [
        {
            'position': i,
            'character': r['character'],
            'unicode': r['unicode'],
            'confidence': float(r['confidence']),
            'bbox': r['bbox']
        }
        for i, r in enumerate(results)
    ]
}

output_path = DATA_DIR / "processed" / "ocr_results.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f"\n✓ Resultados salvos em: {output_path}")
print(f"\nResumo:")
print(f"  Texto: {recognized_text}")
print(f"  Caracteres: {len(results)}")
print(f"  Confiança média: {output_data['average_confidence']:.2%}")